In [192]:
import pandas as pd
import itertools
import math
import time
from collections import Counter

In [2]:
df1=pd.read_json("jsons/dblp-ref-0.json",lines=True)
df2=pd.read_json("jsons/dblp-ref-1.json",lines=True)
df3=pd.read_json("jsons/dblp-ref-2.json",lines=True)
df4=pd.read_json("jsons/dblp-ref-3.json",lines=True)
frames=[df1,df2,df3,df4]

In [176]:
df=pd.concat(frames)
df.head()

,abstract,authors,id,references,title,venue,year
0,The purpose of this study is to develop a lear...,"[Makoto Satoh, Ryo Muramatsu, Mizue Kayama, Ka...",00127ee2-cb05-48ce-bc49-9de556b93346,"[51c7e02e-f5ed-431a-8cf5-f761f266d4be, 69b625b...",Preliminary Design of a Network Protocol Learn...,international conference on human-computer int...,2013
1,This paper describes the design and implementa...,"[Gareth Beale, Graeme Earl]",001c58d3-26ad-46b3-ab3a-c1e557d16821,"[10482dd3-4642-4193-842f-85f3b70fcf65, 3133714...",A methodology for the physically accurate visu...,visual analytics science and technology,2011
2,This article applied GARCH model instead AR or...,"[Altaf Hossain, Faisal Zaman, Mohammed Nasser,...",001c8744-73c4-4b04-9364-22d31a10dbf1,"[2d84c0f2-e656-4ce7-b018-90eda1c132fe, a083a1b...","Comparison of GARCH, Neural Network and Suppor...",pattern recognition and machine intelligence,2009
3,NaN,"[Jea-Bum Park, Byungmok Kim, Jian Shen, Sun-Yo...",00338203-9eb3-40c5-9f31-cbac73a519ec,"[8c78e4b0-632b-4293-b491-85b1976675e6, 9cdc54f...",Development of Remote Monitoring and Control D...,,2011
4,NaN,"[Giovanna Guerrini, Isabella Merlo]",0040b022-1472-4f70-a753-74832df65266,NaN,Reasonig about Set-Oriented Methods in Object ...,,1998


In [177]:
df.drop(['id','references','title','venue'],axis=1,inplace=True)

In [178]:
print("With Nan: ",df.shape)
df.dropna(inplace=True)
print("Without Nan: ",df.shape)

With Nan:  (3079007, 3)
Without Nan:  (2548530, 3)


In [179]:
def handle_abstract(old_abstract):
    if 'Background#R##N#' in old_abstract:
        return(old_abstract[16:])
    return(old_abstract)

In [180]:
df['abstract'] = df['abstract'].apply(handle_abstract)
df=df.sort_values('year')
df=df.reset_index(drop=True)

In [181]:
df.shape

(2548530, 3)

In [182]:
df=df.drop(df[df.authors.map(len)<2].index)
df=df.reset_index(drop=True)

In [183]:
df.shape

(2258334, 3)

In [189]:
def extract_coauthors(df):
    coauthor_list=[]
    for i in range(df.shape[0]):
        if(i%10000==0):
            print(i//10000,end=' ')
        comb=list(itertools.combinations(df.iloc[i].authors, 2))
        coauthor_list.extend(list(map(lambda data: (data,df.iloc[i].year),comb)))

    return coauthor_list

In [190]:
coauthor_list=extract_coauthors(df)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 

In [242]:
author_list=set()
for coauthor,score in coauthor_list:
    author_list.add(coauthor[0])
    author_list.add(coauthor[1])

In [256]:
author_list = list(author_list)
author_dict = {key: '' for key in author_list}

In [257]:
count = 0
for i in range(df.shape[0]):
    if count%100000 == 0:
        print(count//100000,end=' ')
    count+=1
    for author in df.iloc[i].authors:
        author_dict[author] += df.iloc[i].abstract

0 1 2 3 4 5 6 7 

KeyboardInterrupt: 

In [124]:
present_year=df['year'].max()

In [206]:
%%time
coauthor_set = Counter()
for k,v in coauthor_list:
    coauthor_set[tuple(sorted(k))] += 1/math.log(present_year-v+2,2)

CPU times: user 36.8 s, sys: 3.18 s, total: 40 s
Wall time: 47.5 s


In [207]:
coauthor_set=list(coauthor_set.items())

In [220]:
len(coauthor_set)

7499042

In [234]:
main_df = {'Author1':[], 'Author2':[], 'Score':[]}
for coauthor,score in coauthor_set:
    main_df['Author1'].append(coauthor[0])
    main_df['Author2'].append(coauthor[1])
    main_df['Score'].append(score)
main_df=pd.DataFrame(main_df)

In [235]:
main_df['Score']=(main_df['Score']-main_df['Score'].min())/(main_df['Score'].max()-main_df['Score'].min())